# Expérience 5.2 (papier) — Comparer les gaps pour un VAE amorti (MNIST)

Ici, on reproduit l’expérience 5.2 du papier : on entraîne **un VAE complet** (encodeur + décodeur) avec une famille amortie donnée $q_\phi(z|x)$, puis on mesure les trois gaps sur un petit subset fixe.

On compare deux modèles amortis :
- **FFG (Gaussian)** : $q_\phi(z|x)$ est une gaussienne factorisée,
- **Flow** : $q_\phi(z|x)$ est un posterior plus flexible (avec transformations inversibles).
- Pour fashion mnist, on étend au **Contextual Flow**.

La procédure est :
1) On fixe un subset de données (ici 10 points) pour comparer vite et de façon stable.  
2) Pour chaque modèle amorti (FFG puis Flow), on entraîne le VAE sur tout le train set.  
3) Sur le subset, on estime $\log \hat p(x)$ (IWAE et aussi AIS ; on garde le max, comme dans le papier).  
4) On calcule $\mathcal{L}[q]$ : l’ELBO **amorti** avec l’encodeur appris.  
5) On calcule $\mathcal{L}[q^*]$ en faisant une **optimisation locale** de $q$ pour chaque point (on le fait dans deux familles possibles : Gaussien et Flow).  

Ensuite, on déduit les gaps :
- approximation gap : $\log \hat p(x) - \mathcal{L}[q^*]$  
- amortization gap : $\mathcal{L}[q^*] - \mathcal{L}[q]$  
- inference gap : $\log \hat p(x) - \mathcal{L}[q]$

Ce qu’on veut observer : même si le modèle Flow est plus expressif, est-ce que le gain vient surtout de la **réduction de l’approximation gap**, ou est-ce qu’on réduit aussi l’**amortization gap** (donc l’encodeur généralise mieux) ? 

# Import

In [1]:
import os, sys, numpy as np, csv, time
from pathlib import Path
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm

base_dir = Path.cwd().parent
sys.path.insert(0, str(base_dir / 'models'))
sys.path.insert(0, str(base_dir / 'models' / 'utils'))


from vae_2 import VAE
from inference_net import standard
from distributions import Gaussian, Flow
from optimize_local_q import optimize_local_q_dist
from ais3 import test_ais

# Dataset and Device 

In [ ]:
# On charge le dataset MNIST binarisé (train / valid / test) depuis le fichier .npz
data = np.load('binarized_mnist.npz')

# On récupère les trois splits directement depuis les clés du fichier
train_x, valid_x, test_x = data['train_data'], data['valid_data'], data['test_data']

# On fixe les dimensions du problème :
# x_size = 28*28 = 784 pixels, z_size = 50 dimensions latentes (comme dans le papier)
x_size, z_size = 784, 50

print(f" Train {train_x.shape}, Valid {valid_x.shape}, Test {test_x.shape}")

device = torch.device(
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'
)

print(f"Device actif : {device}")

# Définitions des fonctions utiles 
- de train 
- d'évaluations des gaps


In [ ]:
@torch.no_grad()
def estimate_logp_IWAE(model, X, K=10000, batch_size=100):
    # On estime log p(x) avec IWAE
    # torch.no_grad() car on ne garde pas le graphe de gradient, c’est juste de l’évaluation.
    vals = []

    loader = torch.utils.data.DataLoader(
        torch.from_numpy(X).float(), batch_size=batch_size, shuffle=False
    )

    for xb in loader:
        xb = xb.to(device)

        # forward2 renvoie typiquement l'IWAE bound pour k=K
        v, _, _ = model.forward2(xb, k=K)
        vals.append(v.item())

    # On renvoie la moyenne sur tous les batchs
    return float(np.mean(vals))


def estimate_logp_AIS(model, X, K=100, T=500, batch_size=100):
    # On estime log p(x) via AIS 
    # Ici K = nb de particules AIS, T = nb d'intermédiaires
    vals = []

    for i in range(0, len(X), batch_size):
        xb = X[i:i+batch_size]
        try:
            # test_ais renvoie un estimateur de log p(x) 
            est = test_ais(model, xb, xb.shape[0], 0, K, T)

            vals.append(float(est.item() if torch.is_tensor(est) else est))
        except Exception as e:
            print("AIS batch fail:", e)

    return float(np.mean(vals)) if vals else np.nan


@torch.no_grad()
def amortized_elbo(model, X, batch_size=100):
    # On calcule L[q_phi] : l'ELBO amorti du modèle (encoder appris).
    vals = []

    loader = torch.utils.data.DataLoader(
        torch.from_numpy(X).float(), batch_size=batch_size, shuffle=False
    )

    for xb in loader:
        xb = xb.to(device)

        # forward renvoie l'ELBO
        v, _, _ = model.forward(xb, k=1, warmup=1.0)
        vals.append(v.item())

    return float(np.mean(vals))



def locally_optimized_elbo(model, X, q_star_class, n_points=10):
    # On calcule L[q*] : ELBO avec une distribution locale q optimisée par point.
    # Idée : on garde p_theta fixé (le decoder) et on optimise q sur chaque x.
    hyper = model.hyper_params
    vals = []

    # On ne le fait que sur quelques points car c'est coûteux.
    for i in tqdm(range(min(n_points, len(X))), desc=f"q* = {q_star_class.__name__}"):
        # On prend un seul point x (shape [1, x_size])
        x = torch.from_numpy(X[i]).float().view(1, -1).to(device)
        logpost = lambda z: model.logposterior_func2(x=x, z=z)

        # On instancie une q locale de la famille demandée (FFG / Flow / etc.)
        q_local = q_star_class(hyper).to(device)

        # On "warm-start" q_local depuis la q amortie du modèle
        try:
            q_local.load_state_dict(model.hyper_params["q"].state_dict())
        except:
            pass

        # Optimisation locale : on maximise l'ELBO pour ce x en mettant à jour q_local.
        Lqs, _ = optimize_local_q_dist(logpost, hyper, x, q_local)

        # On stocke la valeur finale (ELBO optimisé pour ce point).
        vals.append(float(Lqs.item()))

    # On renvoie la moyenne des L[q*] sur les points testés.
    return float(np.mean(vals))


def build_vae(q_class):
    # On construit un VAE avec une architecture fixe (encoder/decoder MLP).
    # Ici on utilise une sortie encoder de taille 2*z_size (mu + logvar).
    enc_arch = [[x_size,200],[200,200],[200,2*z_size]]
    dec_arch = [[z_size,200],[200,200],[200,x_size]]

    hyper = {
        "x_size": x_size, "z_size": z_size,
        "act_func": F.elu,  # activation (ELU)
        "encoder_arch": enc_arch,
        "decoder_arch": dec_arch,
        "q_dist": standard,  
        "cuda": int(device.type == "mps"),  
        "hnf": 0
    }

    # On crée la distribution variationnelle q (Gaussian, Flow, etc.)
    q = q_class(hyper)
    hyper["q"] = q

    m = VAE(hyper).to(device)
    m.hyper_params = hyper
    return m


def train_model(model, X, epochs=300, batch_size=100, lr=1e-3):
    X_t = torch.from_numpy(X).float()

    loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_t, torch.zeros(len(X))),
        batch_size=batch_size, shuffle=True
    )

    # On optimise à la fois l'encoder (q_dist) et le decoder (generator).
    opt = optim.Adam(
        list(model.q_dist.parameters()) + list(model.generator.parameters()),
        lr=lr
    )

    # Warm-up : on fait monter progressivement le poids du KL au début du training.
    warm_T = 50.0
    global_step = 0

    for ep in range(1, epochs+1):
        for xb, _ in loader:
            xb = xb.to(device)
            global_step += 1

            # warm passe de ~0 à 1 sur les 50 premières itérations 
            warm = min(global_step / warm_T, 1.0)

            # On calcule l'ELBO
            elbo, _, _ = model.forward(xb, k=1, warmup=warm)
            loss = -elbo  # on minimise -ELBO

            opt.zero_grad()
            loss.backward()
            opt.step()
        if ep % 50 == 0:
            print(f"[{ep}/{epochs}] ELBO={elbo.item():.3f}")

    return model

# Définitions de la fonction de run

In [ ]:
def run_exp_5_2(train_x, test_x):
    out_dir = Path("exp52_results_VF")
    out_dir.mkdir(exist_ok=True)

    csv_path = out_dir / "exp52_results_VF.csv"

    if not csv_path.exists():
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "model_family", "q_star", "logp",
                "Lq", "Lq_star",
                "approx_gap", "amort_gap", "infer_gap"
            ])

    # On définit les deux modèles amortis qu’on veut comparer : Gaussian (FFG) vs Flow.
    models = [("FFG", Gaussian), ("Flow", Flow)]

    # On fixe un subset aléatoire pour comparer les modèles équitablement.
    rng = np.random.default_rng(0)
    subset = train_x[rng.choice(len(train_x), size=10, replace=False)]

    # On boucle sur les deux familles : on entraîne un modèle amorti, puis on évalue les gaps.
    for name, q_class in models:

        # On construit le VAE avec la famille de posterior amorti choisie (FFG ou Flow).
        model = build_vae(q_class)

        # On entraîne le modèle sur tout le train set 
        model = train_model(model, train_x, epochs=300)

        # On estime log p(x) sur le subset avec deux estimateurs (IWAE et AIS).
        logp_iwae = estimate_logp_IWAE(model, subset, K=10000)
        logp_ais  = estimate_logp_AIS(model, subset, K=100, T=500)

        # On choisit la meilleure estimation comme dans le papier
        logp = max(logp_iwae, logp_ais if not np.isnan(logp_ais) else -1e9)

        # On calcule l’ELBO amorti L[q_phi] sur le subset.
        Lq = amortized_elbo(model, subset)

        # On calcule l’ELBO localement optimisé avec q* dans la famille Gaussian.
        Lqs_FFG = locally_optimized_elbo(model, subset, Gaussian)

        # On calcule l’ELBO localement optimisé avec q* dans la famille Flow.
        Lqs_Flow = locally_optimized_elbo(model, subset, Flow)

        # On regroupe le calcul des trois gaps (approximation, amortization, total).
        def gaps(logp, Lq, Lqs):
            return logp - Lqs, Lqs - Lq, logp - Lq

        g_FFG  = gaps(logp, Lq, Lqs_FFG)
        g_Flow = gaps(logp, Lq, Lqs_Flow)

        print("\n=== Résultats ===")
        print(f"log p̂(x)     = {logp:.3f}")
        print(f"L[q]         = {Lq:.3f}")
        print(f"L[q*_FFG]    = {Lqs_FFG:.3f}")
        print(f"L[q*_Flow]   = {Lqs_Flow:.3f}")
        print(f"Gaps FFG     = {g_FFG}")
        print(f"Gaps Flow    = {g_Flow}")

        with open(csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, "FFG",  logp, Lq, Lqs_FFG,  *g_FFG])
            writer.writerow([name, "Flow", logp, Lq, Lqs_Flow, *g_Flow])

run_exp_5_2(train_x, test_x)

print(" Résultats sauvés dans exp52_results/exp52_results.csv")